**Todo**

-   doublons ×
-   valeurs vides (whitespace only) ×
-   caractères spéciaux ×
-   url ×
-   espaces multiples ×
-   balises html ×
-   unescape (gérer les \&amp; et autres) ×
-   dates en datetime ×
-   lower ×


In [1]:
import pandas as pd
import re
from html import unescape

## Chargement


In [2]:
fake_df = pd.read_csv('../data/Fake.csv')
true_df = pd.read_csv('../data/True.csv')
fake_df['label'] = 'fake'
true_df['label'] = 'true'
true_df.head()

,title,text,subject,date,label
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",true
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",true
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",true
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",true
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",true


# Exploration


In [3]:
fake_df.describe()

,title,text,subject,date,label
count,23481,23481,23481,23481,23481
unique,17903,17455,6,1681,1
top,MEDIA IGNORES Time That Bill Clinton FIRED His...,,News,"May 10, 2017",fake
freq,6,626,9050,46,23481


In [4]:
true_df.describe()

,title,text,subject,date,label
count,21417,21417,21417,21417,21417
unique,20826,21192,2,716,1
top,Factbox: Trump fills top jobs for his administ...,(Reuters) - Highlights for U.S. President Dona...,politicsNews,"December 20, 2017",true
freq,14,8,11272,182,21417


In [5]:
fake_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23481 entries, 0 to 23480
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    23481 non-null  object
 1   text     23481 non-null  object
 2   subject  23481 non-null  object
 3   date     23481 non-null  object
 4   label    23481 non-null  object
dtypes: object(5)
memory usage: 917.4+ KB


In [6]:
print(fake_df['subject'].value_counts())
print('')
print(true_df['subject'].value_counts())

subject
News               9050
politics           6841
left-news          4459
Government News    1570
US_News             783
Middle-east         778
Name: count, dtype: int64

subject
politicsNews    11272
worldnews       10145
Name: count, dtype: int64


In [7]:
fake_df[fake_df['text'].duplicated()]

,title,text,subject,date,label
9114,WHY THIS BLUE-COLLAR DEMOCRAT STRONGHOLD Count...,AP News The regulars amble in before dawn and...,politics,"Dec 27, 2017",fake
9738,LEFTIST STORE OWNER Makes Video Asking Libs To...,Things didn t go as well as Nicholle had hoped...,politics,"Oct 6, 2017",fake
9942,HILLARY TWEETS MESSAGE In Defense Of DACA…OOPS...,No time to waste we've got to fight with eve...,politics,"Sep 9, 2017",fake
11041,Joe Scarborough BERATES Mika Brzezinski Over “...,,politics,"Apr 26, 2017",fake
11190,WATCH TUCKER CARLSON Scorch Sanctuary City May...,,politics,"Apr 6, 2017",fake
...,...,...,...,...,...
23476,McPain: John McCain Furious That Iran Treated ...,21st Century Wire says As 21WIRE reported earl...,Middle-east,"January 16, 2016",fake
23477,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,21st Century Wire says It s a familiar theme. ...,Middle-east,"January 16, 2016",fake
23478,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...,Patrick Henningsen 21st Century WireRemember ...,Middle-east,"January 15, 2016",fake
23479,How to Blow $700 Million: Al Jazeera America F...,21st Century Wire says Al Jazeera America will...,Middle-east,"January 14, 2016",fake


In [8]:
fake_df[fake_df['title'].duplicated()]

,title,text,subject,date,label
1535,McConnell Says He’ll Obstruct ANY Effort To H...,Siding with Trump s Attorney General Jeff Sess...,News,"May 10, 2017",fake
9942,HILLARY TWEETS MESSAGE In Defense Of DACA…OOPS...,No time to waste we've got to fight with eve...,politics,"Sep 9, 2017",fake
10920,MEDIA IGNORES Time That Bill Clinton FIRED His...,At the recommendation of both Attorney General...,politics,"May 10, 2017",fake
10924,MEDIA IGNORES Time That Bill Clinton FIRED His...,"In its 109-year history, only one F.B.I. direc...",politics,"May 10, 2017",fake
10930,FORMER FBI ASST DIRECTOR: “Jim Comey ‘Danced W...,He threw the reputation of the FBI under the ...,politics,"May 9, 2017",fake
...,...,...,...,...,...
23476,McPain: John McCain Furious That Iran Treated ...,21st Century Wire says As 21WIRE reported earl...,Middle-east,"January 16, 2016",fake
23477,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,21st Century Wire says It s a familiar theme. ...,Middle-east,"January 16, 2016",fake
23478,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...,Patrick Henningsen 21st Century WireRemember ...,Middle-east,"January 15, 2016",fake
23479,How to Blow $700 Million: Al Jazeera America F...,21st Century Wire says Al Jazeera America will...,Middle-east,"January 14, 2016",fake


In [9]:
display(fake_df[fake_df['text'].str.isspace()])
display(true_df[true_df['text'].str.isspace()])

,title,text,subject,date,label
10923,TAKE OUR POLL: Who Do You Think President Trum...,,politics,"May 10, 2017",fake
11041,Joe Scarborough BERATES Mika Brzezinski Over “...,,politics,"Apr 26, 2017",fake
11190,WATCH TUCKER CARLSON Scorch Sanctuary City May...,,politics,"Apr 6, 2017",fake
11225,MAYOR OF SANCTUARY CITY: Trump Trying To Make ...,,politics,"Apr 2, 2017",fake
11236,SHOCKER: Public School Turns Computer Lab Into...,,politics,"Apr 1, 2017",fake
...,...,...,...,...,...
21816,BALTIMORE BURNS: MARYLAND GOVERNOR BRINGS IN N...,,left-news,"Apr 27, 2015",fake
21826,FULL VIDEO: THE BLOCKBUSTER INVESTIGATION INTO...,,left-news,"Apr 25, 2015",fake
21827,(VIDEO) HILLARY CLINTON: RELIGIOUS BELIEFS MUS...,,left-news,"Apr 25, 2015",fake
21857,(VIDEO)ICE PROTECTING OBAMA: WON’T RELEASE NAM...,,left-news,"Apr 14, 2015",fake


,title,text,subject,date,label
8970,Graphic: Supreme Court roundup,,politicsNews,"June 16, 2016",true


In [10]:
display(fake_df[fake_df['title'].str.isspace()])
display(true_df[true_df['title'].str.isspace()])

,title,text,subject,date,label


,title,text,subject,date,label


In [11]:
display(fake_df[fake_df['date'].str.isspace()])
display(true_df[true_df['date'].str.isspace()])

,title,text,subject,date,label


,title,text,subject,date,label


In [12]:
fake_df.isna().sum()

title      0
text       0
subject    0
date       0
label      0
dtype: int64

In [13]:
true_df.isna().sum()

title      0
text       0
subject    0
date       0
label      0
dtype: int64

# Nettoyage


## Supprimer les doublons


In [14]:
clean_fake_df = fake_df.copy()
clean_true_df = true_df.copy()

In [15]:
clean_fake_df = clean_fake_df.drop_duplicates('text')
clean_fake_df.describe()

,title,text,subject,date,label
count,17455,17455,17455,17455,17455
unique,17450,17455,5,1681,1
top,MEDIA IGNORES Time That Bill Clinton FIRED His...,Donald Trump just couldn t wish all Americans ...,News,"May 26, 2016",fake
freq,3,1,9050,35,17455


In [16]:
clean_true_df = clean_true_df.drop_duplicates('text')
clean_true_df.describe()

,title,text,subject,date,label
count,21192,21192,21192,21192,21192
unique,20821,21192,2,716,1
top,Factbox: Trump fills top jobs for his administ...,WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 6, 2017",true
freq,14,1,11214,166,21192


In [17]:
def remove_if_brackets(text):
    if text.strip().startswith('[') and text.strip().endswith(']'):
        return ''
    return text


clean_fake_df['text'] = clean_fake_df['text'].apply(lambda x: remove_if_brackets(x))
clean_fake_df['title'] = clean_fake_df['title'].apply(lambda x: remove_if_brackets(x))

clean_true_df['text'] = clean_true_df['text'].apply(lambda x: remove_if_brackets(x))
clean_true_df['title'] = clean_true_df['title'].apply(lambda x: remove_if_brackets(x))

In [18]:
def normalize_spaces(text):
    return re.sub(r'\s+', ' ', text).strip()


clean_fake_df['text'] = clean_fake_df['text'].apply(lambda x: normalize_spaces(x))
clean_fake_df['title'] = clean_fake_df['title'].apply(lambda x: normalize_spaces(x))

clean_true_df['text'] = clean_true_df['text'].apply(lambda x: normalize_spaces(x))
clean_true_df['title'] = clean_true_df['title'].apply(lambda x: normalize_spaces(x))

In [19]:
def remove_special_characters(text):
    return re.sub(r'[^a-zA-Z0-9\s]', '', text)


clean_fake_df['text'] = clean_fake_df['text'].apply(
    lambda x: remove_special_characters(x)
)
clean_fake_df['title'] = clean_fake_df['title'].apply(
    lambda x: remove_special_characters(x)
)

clean_true_df['text'] = clean_true_df['text'].apply(
    lambda x: remove_special_characters(x)
)
clean_true_df['title'] = clean_true_df['title'].apply(
    lambda x: remove_special_characters(x)
)

In [20]:
def remove_urls(text):
    return re.sub(r'http\S+|www\S+', '', text)


clean_fake_df['text'] = clean_fake_df['text'].apply(lambda x: remove_urls(x))
clean_fake_df['title'] = clean_fake_df['title'].apply(lambda x: remove_urls(x))

clean_true_df['text'] = clean_true_df['text'].apply(lambda x: remove_urls(x))
clean_true_df['title'] = clean_true_df['title'].apply(lambda x: remove_urls(x))

In [21]:
def remove_html_tags(text):
    return re.sub(r'<.*?>', '', text)


clean_fake_df['text'] = clean_fake_df['text'].apply(lambda x: remove_html_tags(x))
clean_fake_df['title'] = clean_fake_df['title'].apply(lambda x: remove_html_tags(x))

clean_true_df['text'] = clean_true_df['text'].apply(lambda x: remove_html_tags(x))
clean_true_df['title'] = clean_true_df['title'].apply(lambda x: remove_html_tags(x))

In [22]:
clean_true_df['text'] = clean_true_df['text'].apply(lambda x: unescape(x))
clean_true_df['title'] = clean_true_df['title'].apply(lambda x: unescape(x))

## Supprimer les chaînes de caractères vides


In [23]:
clean_fake_df = clean_fake_df[clean_fake_df['text'].str.strip().astype(bool)]
clean_fake_df = clean_fake_df[clean_fake_df['title'].str.strip().astype(bool)]
clean_fake_df[clean_fake_df['title'].str.isspace()]

,title,text,subject,date,label


In [24]:
clean_true_df = clean_true_df[clean_true_df['text'].str.strip().astype(bool)]
clean_true_df = clean_true_df[clean_true_df['title'].str.strip().astype(bool)]
clean_true_df[clean_true_df['title'].str.isspace()]

,title,text,subject,date,label


## Normaliser le format des dates


In [25]:
clean_fake_df['date'] = pd.to_datetime(
    clean_fake_df['date'], errors='coerce', format='mixed'
)
clean_true_df['date'] = pd.to_datetime(
    clean_true_df['date'], errors='coerce', format='mixed'
)
clean_fake_df.sample(20)

,title,text,subject,date,label
4123,Ivanka Wont Defend Trumps Rigged Election Clai...,Things started to fall apart faster than ever ...,News,2016-10-19,fake
14550,BAM THIS ONE PICTURE Proves Michael Moore Wron...,Michael Moore seems to think that socialism is...,politics,2016-02-02,fake
17141,FOLLOW THE MONEY MEXICAN GIVEAWAY OF MILLIONS ...,Follow the money on this one Mexico is spendin...,Government News,2015-08-13,fake
10999,TUCKER CARLSON BLASTS Head Of White House Pres...,Tucker Carlson had the Head of the White House...,politics,2017-05-02,fake
15356,VIDEO CHRIS MATTHEWS IS ASTOUNDED WHEN CARLY F...,We re still wondering how Carly Fiorina was re...,politics,2015-08-07,fake
10544,STEVIE WONDER SLAMS Black Lives Matter At MN P...,The music icon spoke at the North Minneapolis ...,politics,2017-06-24,fake
14049,AMBASSADOR CHRIS STEVENS Fianc Speaks Out Abou...,It s amazing that Hillary will likely never be...,politics,2016-04-24,fake
1805,WATCH Reporter Calls Kellyanne Conway And Her ...,Everyone knows by now just how fast and loose ...,News,2017-04-12,fake
2986,Amazon Sells Out Of John Lewis Biographies Aft...,Conservatives are once again on the losing sid...,News,2017-01-16,fake
2629,McConnells Senate Floor Sexism Just Gave Us Th...,The GOP likes to pretend that they have no iss...,News,2017-02-08,fake


In [26]:
print(clean_fake_df['date'].isna().sum())
display(clean_fake_df[clean_fake_df['date'].isna()])

0


,title,text,subject,date,label


## Uniformiser la casse du texte


In [27]:
clean_fake_df['text'] = clean_fake_df['text'].str.lower()
clean_fake_df['title'] = clean_fake_df['title'].str.lower()
clean_fake_df['subject'] = clean_fake_df['subject'].str.lower()

clean_true_df['text'] = clean_true_df['text'].str.lower()
clean_true_df['title'] = clean_true_df['title'].str.lower()
clean_true_df['subject'] = clean_true_df['subject'].str.lower()

clean_true_df.sample(20)

,title,text,subject,date,label
19953,puerto rico opens arms to refugees from irmas ...,san juan puerto rico reuters saddled with eco...,worldnews,2017-09-13,true
13215,jordan sentences syrian militant to death over...,amman reuters a state security court in jorda...,worldnews,2017-12-04,true
1571,proposed healthcare bill may hurt us economy sp,reuters a report from standard poors global ...,politicsnews,2017-09-25,true
15256,kurds displaced by iraq advance fear reprisals...,zinana iraq reuters four hours after first he...,worldnews,2017-11-09,true
6943,trump says taiwan president called me to offer...,washington reuters us presidentelect donald t...,politicsnews,2016-12-03,true
13339,uk must produce credible border plan to unlock...,dublin reuters irish prime minister leo varad...,worldnews,2017-12-01,true
3155,us republican senator wants congress to vote o...,washington reuters us senator jeff flake one ...,politicsnews,2017-06-16,true
13388,trump weighs recognizing jerusalem as israels ...,washington reuters president donald trump is ...,worldnews,2017-11-30,true
11502,court ruling allows syrian youth to bring fami...,berlin reuters a berlin court ruling that per...,worldnews,2017-12-22,true
9025,former ford bush national security adviser sco...,washington reuters brent scowcroft who was na...,politicsnews,2016-06-22,true
